In [1]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Chain import Chain
from Bio.PDB.internal_coords import *
from Bio.PDB.ic_rebuild import write_PDB, IC_duplicate, structure_rebuild_test
from Bio.PDB.SCADIO import write_SCAD
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB.PDBIO import PDBIO
from Bio.SVDSuperimposer import SVDSuperimposer
import numpy as np
import mdtraj as md
import nglview as nv

In [3]:
parser = PDBParser()
files = ['alphaD_minus_cis.pdb',
    'alphaD_plus_cis.pdb',
    'alpha_minus_cis.pdb',
    'alpha_plus_cis.pdb',
    'c7beta_minus_cis.pdb',
    'c7beta_plus_cis.pdb',
    'alphaD_minus_trans.pdb',
    'alphaD_plus_trans.pdb',
    'alpha_minus_trans.pdb',
    'alpha_plus_trans.pdb',
    'c7beta_minus_trans.pdb',
    'c7beta_plus_trans.pdb'
]

angles = np.array([[-82, 82, -61, 63, -135, 128, -82, 83, -59, 59, -82, 105],
                 [175, -176, -50, 49, 74, -71, 174, -177, -64, 64, 80, -185],
                 [0, 0, -11, 11, -3, 2, 180, -180, 176, -177, -177, 178]])

multipliers = np.array([])
angle_dict = {"phi": angles[0], "psi": angles[1], "omega": angles[2]}



 # superpose on fixed reference
sup = SVDSuperimposer()
# sup.set(Ca_triplet_ref, Ca_triplet_xyz)
# sup.run()
# rms = sup.get_rms()
# rot, tran = sup.get_rotran()

# translate xyz only then full coords
# crd_on_ref = dot(crd, rot) + tran

for j in range(len(files)):
    filename = "minima_pdb/" + files[j]
    backbone = parser.get_structure("backbone", "backbone_pdb/gly_backbone.pdb")
    bkb_chain = backbone[0]["A"]
    residues = [res for res in bkb_chain.get_residues()]
    bkb_chain.atom_to_internal_coordinates(verbose=True)

    for i in range(len(residues) - 1):

        r = residues[i]


        if r.internal_coord.get_angle("phi") is not None:
            r.internal_coord.set_angle("phi", angle_dict["phi"][j])
        if r.internal_coord.get_angle("psi") is not None:
            r.internal_coord.set_angle("psi", angle_dict["psi"][j])
        if r.internal_coord.get_angle("omega") is not None:
            r.internal_coord.set_angle("omega", angle_dict["omega"][j])


#         atoms = [atom for atom in r.get_atoms()]
#         atoms_plus = [atom for atom in r_plus.get_atoms()]
#         atoms2 = [atom for atom in r2.get_atoms()]
#         atoms2_plus = [atom for atom in r2_plus.get_atoms()]

    bkb_chain.internal_to_atom_coordinates()

#     traj = md.load_pdb(filename)
#     top = traj.topology
#     residues = [res for res in top.residues]
    for i in range(len(residues) -1):
        r = residues[i]
        r_plus = residues[i+1]
        gly2 = parser.get_structure("glycylglycine", "GLY2.pdb")
        gchain = gly2[0]["A"]
        gres = [res for res in gchain.get_residues()][0]
        moving_coords = [atom.coord for atom in gres.get_atoms()]
        if filename == "minima_pdb/alpha_minus_trans.pdb":
            gly_moving = np.array(moving_coords)[[1, 5, 2]]
            ca_c_coord = np.array([atom.coord for atom in r.get_atoms()])[[2]]
            n_ca_coord = np.array([atom.coord for atom in r_plus.get_atoms()])[[0, 1]]
        elif "trans" in filename:
            gly_moving = np.array(moving_coords)[[0, 1, 5, 2]]
            ca_c_coord = np.array([atom.coord for atom in r.get_atoms()])[[1, 2]]
            n_ca_coord = np.array([atom.coord for atom in r_plus.get_atoms()])[[0, 1]]
        else:
            ca_c_coord = np.array([atom.coord for atom in r.get_atoms()])[[1, 2]]
            n_ca_coord = np.array([atom.coord for atom in r_plus.get_atoms()])[[0]]
            gly_moving = np.array(moving_coords)[[0, 1, 5]]
        fixed = np.concatenate((ca_c_coord, n_ca_coord))
        sup.set(fixed, gly_moving)
        sup.run()
        rms = sup.get_rms()
#         if rms > 0.05:
#             print(filename + ": " + str(rms))
        rot, tran = sup.get_rotran()
        r_atoms = [atom for atom in r.get_atoms()]
        gly_atoms = [atom for atom in gres.get_atoms()]
        r_atoms[3].coord = np.dot(gly_atoms[6].coord, rot) + tran
        
#         if filename == "minima_pdb/alpha_plus_trans.pdb":
#             idxs = [int(k) for k in correct_indices]
#             print(np.array([atom for atom in top.atoms])[idxs])
#             print(np.array([atom for atom in gly2.topology.atoms])[[0, 1, 5]])
#             print(np.array([atom for atom in top.atoms])[indices[3]])
#         gly2.superpose(traj, atom_indices = [0, 1, 5, 2], ref_atom_indices=correct_indices)
#         if np.linalg.norm(gly2.xyz[0, 0] - traj.xyz[0, correct_indices[0]]) > 0.1:
#             print("here")
#             gly2 = md.load_pdb("GLY2.pdb")
#             gly2.superpose(traj, atom_indices = [4, 0, 1], ref_atom_indices=indices[0:3])

#         traj.xyz[0, indices[3]] = gly2.xyz[0, 6]
        

#         if filename == "minima_pdb/alpha_plus_trans.pdb":
#             print(np.linalg.norm(gly2.xyz[0, 0] - traj.xyz[0, correct_indices[0]]))
#             print(np.linalg.norm(gly2.xyz[0, 6] - gly2.xyz[0, 5]))
    write_PDB(backbone, filename)


In [295]:
view = nv.show_file("minima_pdb/alpha_plus_cis.pdb")
view.clear()
view.add_ball_and_stick()
view

NGLWidget()

In [306]:
nv.show_file("GLY2.pdb")

NGLWidget()

In [21]:
import nglview as nv
parser = PDBParser()
backbone = parser.get_structure("backbone", "backbone_pdb/gly_backbone.pdb")
bkb_chain = backbone[0]["A"]
view = nv.show_biopython(bkb_chain)
view.clear()
view.add_ball_and_stick()
view
# bkb_chain.internal_coord.hedra

NGLWidget()

In [29]:
import nglview as nv
view = nv.show_file("c7beta_plus_cis_hn.pdb")
# view.clear()
# view.add_ball_and_stick()
view

NGLWidget()

In [483]:
import os
import subprocess
import mdtraj as md
directory = "minima_pdb/"
for filename in os.listdir(directory):    molecule.save_pdb(file2)

    f = os.path.join(directory, filename)
    # checking if it is a file
    strf = str(f)
    if "_h.pdb" in strf:
        traj = md.load(strf)
        top = traj.topology
#         onswitch = np.zeros(top.n_atoms, dtype=np.bool_)
#         backbone_indices = top.select('backbone')
#         onswitch[[76, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131]] = 1
#         onswitch[backbone_indices] = 1
#         for i in range(top.n_atoms - 1, -1, -1):
#             if not onswitch[i]:
#                 top.delete_atom_by_index(i)
#         new_traj = md.Trajectory(traj.xyz[0, onswitch], top)
#         new_traj.save_pdb(strf.replace("_h", "_nh")
        print(strf + ": [" + ", ".join([str(i) for i in traj.xyz[0, 77]])[:-1] + "], ")

minima_pdb/alphaD_plus_trans_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alpha_plus_trans_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/c7beta_minus_trans_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alpha_plus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/c7beta_plus_trans_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alphaD_plus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alpha_minus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alphaD_minus_trans_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/c7beta_plus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/c7beta_minus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alphaD_minus_cis_h.pdb: [0.0264, -2.7761, 2.590], 
minima_pdb/alpha_minus_trans_h.pdb: [0.0264, -2.7761, 2.590], 


In [486]:
import nglview as nv
view = nv.show_file("minima_pdb/alpha_plus_cis_h.pdb")
view.clear()
view.add_ball_and_stick()
view

NGLWidget()

In [8]:
view = nv.show_file("molecule.pdb")
view.clear()
view.add_ball_and_stick()
view

NGLWidget()

In [2]:
nv.show_file('molecule.pdb')

NGLWidget()